In [2]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import glob
import pyfits
from collections import OrderedDict

In [3]:

#Find this file at : https://github.com/dscolnic/Pantheon/blob/master/lcparam_full_long_zhel.txt
panarr = np.genfromtxt('/lustre/hpc/icecube/mrameez/WDIR/Dipole_JLA/Pantheon/lcparam_full_long_zhel.txt', skip_header=1)
pannames = np.asarray([k.split()[0] for k in open('/lustre/hpc/icecube/mrameez/WDIR/Dipole_JLA/Pantheon/lcparam_full_long_zhel.txt').readlines()[1:]])

#And this one at https://github.com/cmbant/CosmoMC/blob/master/data/jla_lcparams.txt
#Or download from http://supernovae.in2p3.fr/sdss_snls_jla/ReadMe.html
jlarr = np.genfromtxt('/lustre/hpc/icecube/mrameez/WDIR/Dipole_JLA/jla_likelihood_v6/data/jla_lcparams.txt', skip_header=1)
jlnames = np.asarray([k.split()[0] for k in open('/lustre/hpc/icecube/mrameez/WDIR/Dipole_JLA/jla_likelihood_v6/data/jla_lcparams.txt').readlines()[1:]])
plines = open('/lustre/hpc/icecube/mrameez/WDIR/Dipole_JLA/Pantheon/lcparam_full_long_zhel.txt').readlines()[1:]

In [4]:
jlnamesn = np.asarray([k.replace('sn', '').replace('SDSS','') for k in jlnames])#Because pantheon names dont seem to have these prefixes. You can verify they're still the same SN, by comparing the directions

In [5]:
zhdiffs=OrderedDict()
panzs = OrderedDict()
jlzs = OrderedDict()
for i in range(len(jlnamesn)):
    #print jlnamesn[i], pannames[pannames==jlnamesn[i]]
    if jlnamesn[i] in pannames:
        #print jlarr.transpose()[2][i], panarr.transpose()[2][pannames==jlnamesn[i]]
        zhdiffs[jlnamesn[i]] = np.absolute(jlarr.transpose()[2][i] - panarr.transpose()[2][pannames==jlnamesn[i]][0])#the difference between pantheon and jla redshifts (zhel)
        panzs[jlnamesn[i]] = panarr.transpose()[2][pannames==jlnamesn[i]][0]#pantheon zhel
        jlzs[jlnamesn[i]] = jlarr.transpose()[2][i]#jla redshifts
        

In [6]:
#make everything into numpy arrays
jnn = np.asarray(zhdiffs.keys())
jnzd = np.asarray(zhdiffs.values())
jlzarr = np.asarray(jlzs.values())
panzarr = np.asarray(panzs.values())

In [8]:
#print jnn[jnzd>0.0025]
#print len(jnn[jnzd>0.0025])
#Generate the rows of tables 1 and 2 in the arxiv note. Note that the last 2 rows have to be treated differently (prefix sn instead of SDSS)
for snn, zd, jlz, panz in zip(jnn[jnzd>0.0025], jnzd[jnzd>0.0025], jlzarr[jnzd>0.0025], panzarr[jnzd>0.0025]):
    #print snn, zd, jlz, panz
    print 'SDSS'+snn + ' & ', str(jlz),' & ', snn + ' & ', str(panz), ' & ', str(zd), ' & ', str(zd/0.0005) + ' $\sigma$ \\\\' 

SDSS12881 &  0.233  &  12881 &  0.237838  &  0.004838  &  9.676 $\sigma$ \\
SDSS12927 &  0.175  &  12927 &  0.189638  &  0.014638  &  29.276 $\sigma$ \\
SDSS13044 &  0.121  &  13044 &  0.125735  &  0.004735  &  9.47 $\sigma$ \\
SDSS13136 &  0.366  &  13136 &  0.371627  &  0.005627  &  11.254 $\sigma$ \\
SDSS13152 &  0.207  &  13152 &  0.203311  &  0.003689  &  7.378 $\sigma$ \\
SDSS13305 &  0.201  &  13305 &  0.214557  &  0.013557  &  27.114 $\sigma$ \\
SDSS13727 &  0.221  &  13727 &  0.226402  &  0.005402  &  10.804 $\sigma$ \\
SDSS13796 &  0.145  &  13796 &  0.148518  &  0.003518  &  7.036 $\sigma$ \\
SDSS14261 &  0.281  &  14261 &  0.285517  &  0.004517  &  9.034 $\sigma$ \\
SDSS14331 &  0.214  &  14331 &  0.220905  &  0.006905  &  13.81 $\sigma$ \\
SDSS14397 &  0.371  &  14397 &  0.386084  &  0.015084  &  30.168 $\sigma$ \\
SDSS14437 &  0.144  &  14437 &  0.149098  &  0.005098  &  10.196 $\sigma$ \\
SDSS14481 &  0.255  &  14481 &  0.243249  &  0.011751  &  23.502 $\sigma$ \\
SDSS15

In [ ]:
plt.hist(np.log10(zhdiffs.values()), bins=np.linspace(-6, +6, 20))
plt.show()

In [ ]:
for jn in jlnamesn:
    print jn
    if jn in pannames:
        print 'in Pantheon'
        